In [1]:
import streamlit as st
import pandas as pd
import plotly.express as px

In [2]:
# 1. Load and clean the data
@st.cache_data
def load_data():
    df = pd.read_csv('Superstore.csv', encoding='ISO-8859-1')
    df.dropna(inplace=True)
    df['Order Date'] = pd.to_datetime(df['Order Date'])
    df['Ship Date'] = pd.to_datetime(df['Ship Date'])
    df['Profit Margin (%)'] = (df['Profit'] / df['Sales']) * 100
    return df

df = load_data()

2025-05-14 21:55:51.762 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-05-14 21:55:52.125 
  command:

    streamlit run c:\Users\Haithem\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-05-14 21:55:52.133 No runtime found, using MemoryCacheStorageManager


In [3]:
# 2. Sidebar filters
st.sidebar.title("Filters")
regions = st.sidebar.multiselect("Select Region(s):", df['Region'].unique(), default=df['Region'].unique())
categories = st.sidebar.multiselect("Select Category(s):", df['Category'].unique(), default=df['Category'].unique())
date_range = st.sidebar.date_input("Select Order Date Range:",
                                   [df['Order Date'].min(), df['Order Date'].max()])

filtered_df = df[
    (df['Region'].isin(regions)) &
    (df['Category'].isin(categories)) &
    (df['Order Date'] >= pd.to_datetime(date_range[0])) &
    (df['Order Date'] <= pd.to_datetime(date_range[1]))
]

In [4]:
# 3. Title
st.title("📊 Superstore Sales Performance Dashboard")
st.markdown("A dynamic dashboard to analyze sales, profit, and performance metrics from the Superstore dataset.")


DeltaGenerator()

In [5]:
# 4. KPIs
total_sales = filtered_df['Sales'].sum()
total_profit = filtered_df['Profit'].sum()
profit_margin = (total_profit / total_sales) * 100
col1, col2, col3 = st.columns(3)
col1.metric("Total Sales", f"${total_sales:,.2f}")
print()
col2.metric("Total Profit", f"${total_profit:,.2f}")
col3.metric("Profit Margin", f"{profit_margin:.2f}%")

DeltaGenerator()

In [6]:
# Sales Over Time (Fixed)
sales_by_month = (
    filtered_df
    .groupby(filtered_df['Order Date'].dt.to_period("M"))[['Sales']]
    .sum()
    .reset_index()
)
sales_by_month['Order Date'] = sales_by_month['Order Date'].astype(str)

fig1 = px.line(sales_by_month, x='Order Date', y='Sales', title='📈 Monthly Sales Trend')
st.plotly_chart(fig1)

DeltaGenerator()

In [7]:
# Sales by Category
sales_by_category = filtered_df.groupby('Category')['Sales'].sum().reset_index()
fig2 = px.bar(sales_by_category, x='Category', y='Sales', title='📦 Sales by Product Category',
              color='Sales', color_continuous_scale='Blues')
st.plotly_chart(fig2)

# Profit by Region
profit_by_region = filtered_df.groupby('Region')['Profit'].sum().reset_index()
fig3 = px.bar(profit_by_region, x='Region', y='Profit', title='🌍 Profit by Region',
              color='Profit', color_continuous_scale='RdYlGn')
st.plotly_chart(fig3)

# Data Preview (optional)
with st.expander("🔍 View Filtered Data"):
    st.dataframe(filtered_df[['Order Date', 'Region', 'Category', 'Sales', 'Profit', 'Profit Margin (%)']])
